In [2]:
%matplotlib inline
import importlib
from pytorch_utils_oh_2 import *
import IPython

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
MODEL_SAVE_PATH = 'whole_gen_13_only_chars_and_attn'

In [5]:
import pytorch_utils_oh_2; importlib.reload(pytorch_utils_oh_2); from pytorch_utils_oh_2 import *;

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


# Data loading

In [81]:
# all_data = pickle.load(open("data/en_train_fixed_4_sentences.pkl", "rb" ))
all_data = pickle.load(open("data/en_train_fixed_5_manual.pkl", "rb" ))
# all_data_sentence_index = all_data.set_index('sentence_id')

In [83]:
sample_data = all_data.copy()
sample_data = sample_data[sample_data['class'] != 'NOT_CHANGED']
sample_data = sample_data[sample_data['class'] != 'MANUAL']
sample_data = pd.concat([sample_data, all_data[all_data['class_org'] == 'PLAIN'].sample(1000000)])
# sample_data = sample_data[sample_data['class'] != 'ELECTRONIC']
sample_data = sample_data.reset_index(drop=True)
print("Data rows: {},  (dropped rows: {})".format(len(sample_data), len(all_data)-len(sample_data)))
#del(all_data)

Data rows: 1654333,  (dropped rows: 8263859)


In [84]:
sample_data.sample(10)

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence
679572,203927,17,NOT_CHANGED,affluent,affluent,PLAIN,NaN,hudnut 's beauty products were sold in departm...
166155,190828,1,NUMBERS,.256,point two five six,DECIMAL,"[46, 5, 14, 20, 0]",the <SAMPLE> newton and the 6 are not intercha...
1085611,725759,13,NOT_CHANGED,receive,receive,PLAIN,NaN,if they handed to rose over to the correct wom...
519840,594469,2,LETTERS,F.C.,f c,LETTERS,"[37, 21, 0]",cardiff city <SAMPLE> 2009 - 11 - 28 .
309072,356414,1,LETTERS,PS,p s,LETTERS,"[24, 17, 0]",""" <SAMPLE> e psd rejeitam proposta para reconh..."
1047151,354179,10,NOT_CHANGED,is,is,PLAIN,NaN,pines of rome ( italian : pini di roma ) <SAMP...
1456288,473851,8,NOT_CHANGED,Mockingbird,Mockingbird,PLAIN,NaN,"isbn 0 - 8071 - 0390 - x "" to kill a <SAMPLE> ..."
659711,652395,4,NOT_CHANGED,Scott,Scott,PLAIN,NaN,sammon wrote that ridley <SAMPLE> thought it w...
53348,61430,5,NUMBERS,2009,two thousand nine,DATE,"[5, 8, 15, 0]","brandeis , magdalene ( spring <SAMPLE> ) ."
482564,552884,16,LETTERS,PDF,p d f,LETTERS,"[24, 26, 37, 0]",""" wealth and happiness revisited growing wealt..."


In [91]:
del(all_data)

In [92]:
categories_all = sorted(sample_data["class"].unique())
print(categories_all)
print(len(categories_all))
categories_index = dict((c, i) for i, c in enumerate(categories_all))

['ELECTRONIC', 'LETTERS', 'NOT_CHANGED', 'NUMBERS', 'PLAIN', 'VERBATIM']
6


In [93]:
sample_data.sample(2)

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence
1300719,547543,9,NOT_CHANGED,Literary,Literary,PLAIN,NaN,manchester is also the recipient of the abraha...
1126530,505044,1,NOT_CHANGED,Sugiura,Sugiura,PLAIN,NaN,ken <SAMPLE> ( 2008 - 12 - 19 ) .


### Utils stuff

In [12]:
chars_normal, chars_normal_index = load_characters_pkl('data/en_features/chars_normal.pkl')
print(''.join(chars_normal))

<SOS><EOS>☒ !"#$%&'(),-./0123456789:;ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz~£¥ª²³µº¼½¾éɒʻˈΩμ—€⅓⅔⅛


In [90]:
chars_after = sorted(list(set(''.join(list(sample_data['after'].unique())).lower())))
chars_after = [SOS_TOKEN, EOS_TOKEN, UNKNOWN_CHAR] + chars_after
chars_after_index = dict((c, i) for i, c in enumerate(chars_after))
print(''.join(chars_after))

<SOS><EOS>☒ 'abcdefghijklmnopqrstuvwxyzé


chars_after, chars_after_index = load_characters_pkl('data/en_features/chars_after_1.pkl')
print(''.join(chars_after))

In [16]:
wv_vecs, wv_words, wv_idx = load_glove('/home/ohu/koodi/data/glove_wordvec/glove.6B.50d.txt')

### After words handling

In [28]:
sos_tensor = torch.zeros(1, 1, len(chars_after_index))
sos_tensor[0, 0, chars_after_index[SOS_TOKEN]] = 1
sos_tensor.size()

torch.Size([1, 1, 32])

### More balanced sample

In [145]:
balanced_data_classes_select = list(sample_data.groupby('class'))

balanced_data_accessed_counter = 0 
balanced_data_randomize_freq = False
balanced_data_length = 0

def balanced_data_randomize_org(max_len=30000):
    global balanced_data, balanced_data_length, balanced_data_accessed_counter, balanced_data_randomize_freq
    balanced_data = pd.concat([v.sample(min(max_len, len(v))) for k, v in balanced_data_classes_select])
    balanced_data = pd.concat([balanced_data, sample_data[sample_data['class']=='NOT_CHANGED'].sample(int(max_len*2))])
    balanced_data_length = len(balanced_data)
    balanced_data_randomize_freq = balanced_data_length * 0.2
    balanced_data_accessed_counter = 0
balanced_data_randomize = balanced_data_randomize_org

def balanced_data_sample_row():
    global balanced_data_accessed_counter
    global balanced_data_last_sample
    balanced_data_accessed_counter += 1
    if balanced_data_randomize_freq and balanced_data_accessed_counter > balanced_data_randomize_freq:
        balanced_data_randomize()
    balanced_data_last_sample = balanced_data.iloc[random.randint(1, balanced_data_length-1)]
    return balanced_data_last_sample
    
balanced_data_randomize()

In [146]:
#all_data.groupby('class')['class'].count()
#sample_data.groupby('class')['class'].count()
balanced_data.groupby('class')['class'].count()

class
ELECTRONIC      4964
LETTERS        30000
NOT_CHANGED    90000
NUMBERS        30000
PLAIN          30000
VERBATIM       11741
Name: class, dtype: int64

In [147]:
balanced_data_sample_row()

sentence_id                                               376723
token_id                                                      13
class                                                NOT_CHANGED
before                                           amitochondriate
after                                            amitochondriate
class_org                                                  PLAIN
a_word_ind                                                   NaN
sentence       " the chimeric eukaryote : origin of the nucle...
Name: 934523, dtype: object

### Samples

In [98]:
def get_random_sample():
    sample_row = balanced_data_sample_row()   
    return sample_row['before'], sample_row['after'].lower(), sample_row['class'], sample_row['sentence'].split(' ')
            
def tmp():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    print(s_class, ':', s_bef, '->', s_aft)
    print(s_sentence)
    print(string_to_tensor(s_bef, chars_normal_index).shape)
tmp()

LETTERS : U.S. -> u s
['22', '<SAMPLE>', 'at', '675', '.']
torch.Size([1, 5, 104])


In [60]:
%%time
get_random_sample()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 291 µs


('.',
 '.',
 'NOT_CHANGED',
 ['in', '1925', 'william', 'barclay', 'peat', '&', 'co', '<SAMPLE>'])

# Model functions

In [40]:
use_cuda = True

In [103]:
MAX_ATTENTION_LENGTH = 30

In [104]:
tmp = sample_data[sample_data['before'].str.len()>MAX_ATTENTION_LENGTH]
len(tmp)
tmp.sample(2)
# tmp[~tmp['before'].str.contains('/')].sample(2)

751

,sentence_id,token_id,class,before,after,class_org,a_word_ind,sentence
19757,22689,10,ELECTRONIC,http://www.kiwiaircraftimages.com/mustang.html,h t t p colon slash slash w w w dot k i w i a ...,ELECTRONIC,"[45, 30, 30, 24, 129, 101, 101, 52, 52, 52, 74...","missing plane , evening post , 5 september 194..."
9517,11138,2,ELECTRONIC,http://www.jaabc.com/brcv14n1preview.htmlHarvard,h t t p colon slash slash w w w dot j a a b c ...,ELECTRONIC,"[45, 30, 30, 24, 129, 101, 101, 52, 52, 52, 74...","retrieved from <SAMPLE> business review , hbr ..."


### Encoder

In [105]:
class EncoderRNN(nn.Module):
    def __init__(self, words_input_size, chars_input_size, words_hidden_size, chars_hidden_size,
                 words_layers=1, chars_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.words_layers = words_layers
        self.chars_layers = chars_layers
        self.words_hidden_size = words_hidden_size
        self.chars_hidden_size = chars_hidden_size
        self.hidden_size = words_hidden_size + chars_hidden_size

        self.rnn_words = nn.GRU(words_input_size, words_hidden_size // 2, words_layers,
                                 batch_first=True, bidirectional=True)

        self.rnn_chars = nn.GRU(chars_input_size, chars_hidden_size // 2, chars_layers,
                                 batch_first=True, bidirectional=True)
        
    def forward(self, word_vectors, string_tensor, hidden = None, init_hidden = True):
        if init_hidden:
            hidden_words, hidden_chars = self.init_hidden()
        
        all_outputs_words, hidden_words = self.rnn_words(word_vectors, hidden_words)
        output_words = hidden_words.view(1, -1)
        
        all_outputs_chars, hidden_chars = self.rnn_chars(string_tensor, hidden_chars)
        output_chars = hidden_chars.view(1, -1)
        
        #hidden_states_cat = Variable(torch.zeros(MAX_ATTENTION_LENGTH, self.hidden_size)).cuda()
        #for ei in range(min(MAX_ATTENTION_LENGTH, len(string_tensor[0]))):
        #    hidden_states_cat[ei] = torch.cat((output_words, all_outputs_chars[0, ei].view(1,-1)), 1)

        all_outputs_chars_padded = Variable(torch.zeros(MAX_ATTENTION_LENGTH, self.chars_hidden_size)).cuda()
        att_length = min(len(all_outputs_chars[0]), MAX_ATTENTION_LENGTH-1)
        all_outputs_chars_padded[0:att_length] = all_outputs_chars[0][0:att_length]
        
        output = torch.cat((output_words, output_chars), 1)

        #output = torch.cat((output_words, output_chars), 1)
        
        #return output, all_outputs_chars
        return output, all_outputs_chars_padded

    def init_hidden(self):
        var1 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var2 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        
        var1 = var1.cuda(); var2 = var2.cuda()
        return (var1, var2)
    
    
encoder_rnn = EncoderRNN(words_input_size=wv_vecs.shape[-1], chars_input_size=len(chars_normal),
                         words_hidden_size=128, chars_hidden_size=512,
                         words_layers=1, chars_layers=1).cuda()
encoder_rnn

EncoderRNN (
  (rnn_words): GRU(50, 64, batch_first=True, bidirectional=True)
  (rnn_chars): GRU(104, 256, batch_first=True, bidirectional=True)
)

In [106]:
def test_encoder_single_sample():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    #s_bef, s_aft, s_class, s_sentence = get_random_sample(True)
    print(s_bef)
    
    words_t = Variable(words_to_word_vectors_tensor(s_sentence, wv_vecs, wv_idx)).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    return encoder_rnn(words_t, string_t), s_bef
    
(tmp_encoder_output, tmp_encoder_outputs), tmp = test_encoder_single_sample()
tmp
tmp_encoder_output.size()
tmp_encoder_outputs.size()

rough


'rough'

torch.Size([1, 640])

torch.Size([30, 512])

### Decoder

In [110]:
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, chars_encoded_size,
                 n_layers=1, dropout_p=0.1, max_length=MAX_ATTENTION_LENGTH):
        super(DecoderRNN, self).__init__()
        
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.max_length = max_length
        
        self.emb_lin = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.attn = nn.Linear(self.hidden_size+self.hidden_size, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size+chars_encoded_size, self.hidden_size)
        
        #self.module_attn = torch.nn.ModuleList([self.emb_lin, self.dropout, self.attn, self.attn_combine])
        
        
        self.rnn = nn.GRU(hidden_size, hidden_size, n_layers, batch_first=True, bidirectional=False)
        self.lin_out = nn.Linear(hidden_size, input_size)
        
        #self.module_rnn = torch.nn.ModuleList([self.rnn, self.lin_out])

    def forward(self, last_input, hidden, encoder_outputs):
        embedded = self.emb_lin(last_input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        embedded = embedded[0]
                
        attn_weights = F.softmax(self.attn(torch.cat((embedded, hidden), 1)))
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        #return embedded, attn_applied
        
        rnn_input = torch.cat((embedded, attn_applied[0]), 1)
        rnn_input = self.attn_combine(rnn_input).unsqueeze(0)
        rnn_input = F.relu(rnn_input)
    
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        output = F.log_softmax(self.lin_out(output[0]))
        
        return output, hidden[0], attn_weights
    
    def init_rest_hidden(self, input_var):
        if self.n_layers > 1:
            hid_var = Variable(torch.zeros(self.n_layers - 1, 1, self.hidden_size)).cuda()
            res = torch.cat((input_var, hid_var), 0)
            return res
        else:
            return input_var
        
    def mods_split(self):
        mods = list(decoder_rnn.modules())[1:]
        for gru_index, mod in enumerate(mods):
            #print(mod)
            if type(mod) == torch.nn.modules.rnn.GRU:
                break
        return mods[:gru_index], mods[gru_index:]
        
    def mods_attn(self):
        return self.mods_split()[0]
        
    def mods_gru(self):
        return self.mods_split()[1]

decoder_rnn = DecoderRNN(input_size=len(chars_after), hidden_size=tmp_encoder_output.size()[1],
                         chars_encoded_size=tmp_encoder_outputs.size()[1], n_layers=1)
decoder_rnn = decoder_rnn.cuda()
decoder_rnn

tmp = decoder_rnn(Variable(sos_tensor).cuda(), tmp_encoder_output, tmp_encoder_outputs)
#tmp
[v.size() for v in tmp]

DecoderRNN (
  (emb_lin): Linear (32 -> 640)
  (dropout): Dropout (p = 0.1)
  (attn): Linear (1280 -> 30)
  (attn_combine): Linear (1152 -> 640)
  (rnn): GRU(640, 640, batch_first=True)
  (lin_out): Linear (640 -> 32)
)

[torch.Size([1, 32]), torch.Size([1, 640]), torch.Size([1, 30])]

In [111]:
tmp = [{'params': mod.parameters(), 'lr': 3} for mod in decoder_rnn.mods_attn()]
tmp += [{'params': mod.parameters()} for mod in decoder_rnn.mods_gru()]
tmp.append(
    {'params': encoder_rnn.parameters()}
)
tmp

[{'lr': 3, 'params': <generator object Module.parameters at 0x7fc10bbee4c0>},
 {'lr': 3, 'params': <generator object Module.parameters at 0x7fc10bbee990>},
 {'lr': 3, 'params': <generator object Module.parameters at 0x7fc10bbee780>},
 {'lr': 3, 'params': <generator object Module.parameters at 0x7fc10bbee6d0>},
 {'params': <generator object Module.parameters at 0x7fc10bbee8e0>},
 {'params': <generator object Module.parameters at 0x7fc10d5eaba0>},
 {'params': <generator object Module.parameters at 0x7fc10d5eaf68>}]

# Training etc

### Accuracy

In [118]:
def test_model_single_sample(model=None, return_more=False, sample=False):
    decoder_rnn.eval()
    encoder_rnn.eval()
    
    if not sample:
        sample = get_random_sample()
    s_bef, s_aft, s_class, s_sentence = sample
        
    words_t = Variable(words_to_word_vectors_tensor(s_sentence, wv_vecs, wv_idx)).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    encoder_output, encoder_outputs = encoder_rnn(words_t, string_t)
    
    decoder_input = Variable(sos_tensor).cuda()
    decoder_hidden = encoder_output
    
    decoded_output = []
    decoder_attns_arr = []
    max_length = 20
    for _ in range(max_length):
        decoder_output, decoder_hidden, decoder_attns = decoder_rnn(decoder_input, decoder_hidden, encoder_outputs)
        decoder_attns_arr.append(decoder_attns)
        #return decoder_output

        topv, topi = decoder_output.data.topk(1)
        char_index = topi[0][0]
        char = chars_after[char_index] # Use own prediction as next input
                
        if char == EOS_TOKEN:
            break

        decoded_output.append(char)
                
        decoder_input = torch.zeros(1, 1, len(chars_after_index))
        decoder_input[0, 0, char_index] = 1
        decoder_input = Variable(decoder_input).cuda()
    
    output = ''.join(decoded_output)
    sample_target = s_aft
    
    decoder_rnn.train()
    encoder_rnn.train()
    
    if return_more:
        return output, decoded_output, decoder_attns_arr, sample
    
    return output, output, sample_target, sample
    
tmp = test_model_single_sample(None)
tmp

('iiiissssssssssssssss',
 'iiiissssssssssssssss',
 'mister',
 ('mr',
  'mister',
  'PLAIN',
  ['<SAMPLE>', 'lucton', "'s", 'freedom', '(', 'hardcover', ')', '.']))

In [119]:
def print_local_wrong_predictions(max_results=10):
    arr = get_some_wrong_predictions(None, test_model_single_sample, max_iterations=10000, max_results=max_results)
    for sample, predict, output in arr:
        s_bef, s_aft, s_class, s_sentence = sample
        print("{:<14} => {:<14} || {} \n{:>17} {}".format(s_bef, predict, s_aft, '', s_sentence, ))
print_local_wrong_predictions(2)

1972           => iiiissssssssssssssss || nineteen seventy two 
                  ['in', '<SAMPLE>', ',', 'he', 'joined', 'hk', 'jesenice', '.']
-              => iiiissssssssssssssss || to 
                  ['the', 'practical', 'upper', 'limit', 'for', 'wooden', 'constructions', 'fast', 'and', 'maneuverable', 'enough', 'for', 'warfare', 'was', 'around', '25', '<SAMPLE>', '30', 'oars', 'per', 'side', '.']


In [120]:
%%time
test_model_accuracy(encoder_rnn, test_model_single_sample, n_sample=100)

Accuracy: 0.00% (       0/     100)
CPU times: user 2.11 s, sys: 20 ms, total: 2.13 s
Wall time: 2.13 s


0.0

### Training

In [167]:
def train(s_bef, s_aft, s_sentence, optimizer, loss_function,
          use_teacher_forcing, max_length=20):
    
    words_t = Variable(words_to_word_vectors_tensor(s_sentence, wv_vecs, wv_idx)).cuda() 
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    target_arr = [chars_after_index[c] for c in list(s_aft)]
    optimizer.zero_grad()
    loss = 0
    
    
    encoder_output, encoder_outputs = encoder_rnn(words_t, string_t)
    
    
    decoder_hidden = encoder_output
    decoder_input = Variable(sos_tensor).cuda()

    decoded_output = []
    for i in range(len(target_arr)):
        decoder_output, decoder_hidden, decoder_attns = decoder_rnn(decoder_input, decoder_hidden, encoder_outputs)

        decoder_target_i = target_arr[i]
        decoder_target_i = Variable(torch.LongTensor([decoder_target_i])).cuda()
        loss += loss_function(decoder_output, decoder_target_i)
        
        topv, topi = decoder_output.data.topk(1)
        char_index = topi[0][0]
        char = chars_after[char_index] # Use own prediction as next input
        decoded_output.append(char)
        
        if use_teacher_forcing:
            char_index = target_arr[i] # replace input with right target
        else:
            # use output normally as input 
            if char == EOS_TOKEN:
                break
                
        decoder_input = torch.zeros(1, 1, len(chars_after_index))
        decoder_input[0, 0, char_index] = 1
        decoder_input = Variable(decoder_input).cuda()
        
    if decoded_output[-1] == EOS_TOKEN:
        decoded_output = decoded_output[:-1]
        
    loss.backward()

    return ''
    # not used yet
    # https://github.com/pytorch/examples/blob/master/word_language_model/main.py
    if clip_parameters_value:
        for m in [decoder_rnn, encoder_rnn]:
            torch.nn.utils.clip_grad_norm(m.parameters(), clip_parameters_value)
    
    optimizer.step()

    return ''.join(decoded_output), (loss.data[0] / len(target_arr))


In [209]:
train_iterations(n_iters=1, print_every=1, lr=0.0001)

ValueError: not enough values to unpack (expected 2, got 0)

In [210]:
parameters = encoder_rnn.parameters()
parameters = list(filter(lambda p: p.grad is not None, parameters)) 

In [211]:
parameters[0].grad.abs().max()

Variable containing:
1.00000e-02 *
  8.0750
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [213]:
tmp = max(p.grad.data.abs().max() for p in parameters)
tmp

0.0807504802942276

In [219]:
0.1/tmp

0.3423733531166572

In [217]:
tmp = 0 
for p in parameters:
    param_norm = p.grad.data.norm(2)
    tmp += param_norm ** 2

In [218]:
tmp

0.29207880546102816

In [216]:
parameters[0].grad[0][0]

Variable containing:
1.00000e-05 *
  7.9407
[torch.cuda.FloatTensor of size 1 (GPU 0)]

In [199]:
torch.ones(1).mul_(0.1)


 0.1000
[torch.FloatTensor of size 1]

In [136]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0.5,
                     print_every=10000, plot_every=1000):
    global optimizer
    start = time.time()
    
    decoder_rnn.train()
    encoder_rnn.train()

    current_loss = 0
    current_loss_iter = 0

    tmp = [{'params': mod.parameters(), 'lr': (lr/10)} for mod in decoder_rnn.mods_attn()]
    tmp += [{'params': mod.parameters()} for mod in decoder_rnn.mods_gru()]
    tmp.append(
        {'params': encoder_rnn.parameters()}
    )
    optimizer = torch.optim.Adam(tmp, lr=lr)
    
    loss_function = nn.NLLLoss()
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        s_bef, s_aft, s_class, s_sentence = get_random_sample()
        
        result, loss = train(s_bef=s_bef, s_aft=s_aft, s_sentence=s_sentence,
                             optimizer=optimizer,
                             loss_function=nn.NLLLoss(), use_teacher_forcing=use_teacher_forcing,
                             max_length=40 )
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            correct = '✓' if result == s_aft else "✗: {}".format(s_aft)
            
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} -> {} ({}) {}".format(
                      model_training.iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss,
                      s_bef, result, correct, teacher_forcing_str))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 0
            
        if model_training.iterations % 50000 == 0 or model_training.iterations == 10:
            model_training.save_models()
            acc = test_model_accuracy(encoder_rnn, test_model_single_sample)
            model_training.accuracy.append(acc)
    
    # test_model_accuracy(model, n_sample=10000)

In [137]:
model_training = ModelTraining(MODEL_SAVE_PATH, [encoder_rnn, decoder_rnn])

Save path: data/models/whole_gen_13_only_chars_and_attn


In [138]:
train_iterations(n_iters=50, print_every=9, lr=0.0001)

     9  18% (   0m 0s)   3.436   |   3.42: & -> iio (✗: and) (forcing)
Saved model to data/models/whole_gen_13_only_chars_and_attn/10_(EncoderRNN/DecoderRNN)
Accuracy: 0.00% (       0/   10000)
    18  36% (  3m 31s)   3.420   |   3.39: - -> ii (✗: to) 
    27  54% (  3m 31s)   3.400   |   3.32: 1958 -> iiooooeoeeeooooeooeo (✗: nineteen fifty eight) (forcing)
    36  72% (  3m 31s)   3.378   |   3.26: 18 April 1840 -> iieeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee (✗: the eighteenth of april eighteen forty) 
    45  90% (  3m 32s)   3.359   |   3.23: rBST -> ieeeeee (✗: r b s t) 


In [139]:
train_iterations(n_iters=(1000-model_training.iterations), print_every=500, lr=0.0001)

   550  53% (  0m 14s)   2.992   |   3.12: ICC -> ateee (✗: i c c) 


In [140]:
optimizer.param_groups[0]['lr']
optimizer.param_groups[6]['lr']

1e-05

0.0001

In [148]:
train_iterations(n_iters=9000, lr=0.0001, print_every=1000)

  2000  11% (  0m 26s)   2.733   |   2.97: her -> ton (✗: her) 
  3000  22% (  0m 53s)   2.622   |   0.83: & -> and (✓) 
  4000  33% (  1m 21s)   2.491   |   2.09: May 2010 -> t n teente n e (✗: may twenty ten) (forcing)
  5000  44% (  1m 51s)   2.339   |   2.45: Vol -> aonina (✗: volume) (forcing)
  6000  56% (  2m 18s)   2.206   |   0.30: & -> and (✓) (forcing)
  7000  67% (  2m 45s)   2.048   |   0.72: - -> to (✓) (forcing)
  8000  78% (  3m 14s)   1.889   |   1.55: January 2015 -> aaadnne twenty tintyen (✗: january twenty fifteen) (forcing)
  9000  89% (  3m 41s)   1.737   |   2.25: Clitheroe -> cetheeter (✗: clitheroe) 
 10000 100% (  4m 11s)   1.748   |   3.67: up -> t  (✗: up) (forcing)


In [149]:
train_iterations(n_iters=90000, print_every=10000)

 20000  11% (  4m 50s)   0.399   |   0.06: Realising -> realizing (✓) 
 30000  22% (  9m 36s)   0.262   |   0.02: side -> side (✓) (forcing)
 40000  33% ( 14m 32s)   0.225   |   0.00: vol -> volume (✓) (forcing)
 50000  44% ( 19m 19s)   0.213   |   0.11: I. -> i (✓) 
Saved model to data/models/whole_gen_13_only_chars_and_attn/50000_(EncoderRNN/DecoderRNN)
Accuracy: 0.61% (      61/   10000)
 60000  56% ( 27m 34s)   0.165   |   0.00: Machine -> machine (✓) (forcing)
 70000  67% (  32m 4s)   0.198   |   0.00: rock -> rock (✓) (forcing)
 80000  78% ( 36m 36s)   0.226   |   0.01: : -> to (✓) (forcing)
 90000  89% (  41m 7s)   0.149   |   0.00: - -> to (✓) (forcing)
100000 100% ( 45m 38s)   0.150   |   0.01: st -> saint (✓) (forcing)
Saved model to data/models/whole_gen_13_only_chars_and_attn/100000_(EncoderRNN/DecoderRNN)
Accuracy: 0.64% (      64/   10000)


In [150]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

110000   5% (  4m 26s)   0.209   |   0.00: for -> for (✓) 
120000  10% (  8m 59s)   0.604   |   3.36: 31 July 2008 -> thei t twalatlousalaousawenuloulanente ttt  (✗: the thirty first of july two thousand eight) 
130000  15% ( 13m 35s)   0.499   |   0.01: PDF -> p d f (✓) (forcing)
140000  20% (  18m 9s)   0.390   |   0.00: white -> white (✓) (forcing)
150000  25% ( 22m 42s)   0.333   |   1.49: 453 -> four  undr d niv y  oi d (✗: four hundred fifty three) (forcing)
Saved model to data/models/whole_gen_13_only_chars_and_attn/150000_(EncoderRNN/DecoderRNN)
Accuracy: 0.00% (       0/   10000)
160000  30% ( 30m 34s)   0.329   |   0.01: R. -> r (✓) 
170000  35% (  35m 8s)   0.903   |   0.00: & -> and (✓) (forcing)
180000  40% ( 39m 49s)   0.468   |   0.25: demobilisation -> domobilization (✗: demobilization) (forcing)
190000  45% ( 44m 19s)   0.669   |   0.00: a -> a (✓) (forcing)
200000  50% ( 48m 56s)   0.576   |   0.60: tumours -> tumurs (✗: tumors) (forcing)
Saved model to data/models/wh

IndexError: list index out of range

In [153]:
[p for p in decoder_rnn.parameters()]

[Parameter containing:
   nan   nan   nan  ...    nan   nan   nan
   nan   nan   nan  ...    nan   nan   nan
   nan   nan   nan  ...    nan   nan   nan
        ...          ⋱          ...       
   nan   nan   nan  ...    nan   nan   nan
   nan   nan   nan  ...    nan   nan   nan
   nan   nan   nan  ...    nan   nan   nan
 [torch.cuda.FloatTensor of size 640x32 (GPU 0)], Parameter containing:
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan

In [154]:
state_dict_path = 'data/models/whole_gen_13_only_chars_and_attn/200000_'

decoder_rnn.load_state_dict(torch.load(state_dict_path + 'DecoderRNN'))
encoder_rnn.load_state_dict(torch.load(state_dict_path + 'EncoderRNN'))

In [155]:
[p for p in decoder_rnn.parameters()]

[Parameter containing:
  0.0160 -0.0180  0.0727  ...  -0.0082 -0.1126  0.1833
  0.0362  0.1504 -0.1483  ...   0.0192  0.0254 -0.1061
  0.0463 -0.0354  0.0112  ...   0.0652  0.0557  0.0154
           ...             ⋱             ...          
  0.0565 -0.0500 -0.1276  ...   0.1339 -0.0295 -0.1846
  0.0903 -0.0143  0.0025  ...   0.0004  0.2131 -0.0659
 -0.1217  0.1283  0.1447  ...  -0.1420 -0.0426 -0.0893
 [torch.cuda.FloatTensor of size 640x32 (GPU 0)], Parameter containing:
 -0.0050
 -0.0018
  0.1565
 -0.0335
 -0.0937
  0.0640
 -0.1053
  0.0412
  0.0524
  0.0312
 -0.0904
 -0.0394
 -0.0696
  0.0449
  0.1610
 -0.0237
  0.0710
  0.0841
 -0.0801
 -0.0465
 -0.0946
 -0.0561
 -0.0086
  0.1023
  0.0728
  0.0560
 -0.0618
 -0.1034
  0.0572
 -0.0217
  0.0586
  0.1533
  0.0727
 -0.0962
  0.0953
  0.0851
 -0.0310
 -0.0842
 -0.0061
 -0.1213
  0.1314
 -0.1919
 -0.0804
  0.0349
 -0.1110
  0.0452
  0.0658
  0.0664
 -0.0486
  0.0106
  0.0187
 -0.1310
  0.1380
 -0.1122
  0.0577
  0.1361
  0.0823
 -0.009

In [158]:
clip_parameters_value = 0.2

In [161]:
train_iterations(n_iters=10000, lr=0.001, print_every=1000)

231040  10% (  1m 54s)   0.563   |   0.00: European -> european (✓) 
232040  20% (  2m 31s)   0.573   |   0.52: Nxe -> n e e (✗: n x e) (forcing)
233040  30% (   3m 8s)   0.597   |   0.00: many -> many (✓) 
234040  40% (  3m 43s)   0.590   |   0.00: M. -> m (✓) (forcing)
235040  50% (  4m 19s)   0.490   |   0.00: tax -> tax (✓) 
236040  60% (  4m 53s)   0.503   |   0.00: done -> done (✓) 
237040  70% (  5m 30s)   0.506   |   0.00: Museum -> museum (✓) 
238040  80% (   6m 8s)   0.550   |   0.00: CSE -> c s e (✓) (forcing)
239040  90% (  6m 42s)   0.499   |   0.00: i -> i (✓) 
240040 100% (  7m 23s)   0.585   |   3.94: 22 13 7 2 42-14 464 -> thtee tof tot tofte tof four tou tou tou t tou t tot tou tou t tot toft tote  t th  (✗: two two sil one three sil seven sil two sil four two sil one four sil four six four) 


In [162]:
print_local_wrong_predictions()

Madison        => madisonsdosadiossand || madison 
                  ['"', 'feature', 'detail', 'report', 'for', ':', 'etlan', '(', '<SAMPLE>', 'county', ',', 'virginia', ')', '"', '.']
&              => anddodreddreddredy o || and 
                  ['recorded', 'at', 'bonnie', 'wee', 'studios', ',', 'scotland', ';', 'windmill', 'lane', ',', 'dublin', ';', 'conway', ',', 'westlake', '<SAMPLE>', 'track', 'records', ',', 'los', 'angeles', 'california', '.']
behaviour      => behaviory threen hte || behavior 
                  ['in', 'guido', ',', 'lefevre', 'and', 'lydgate', "troilus'", 'killer', "'s", '<SAMPLE>', 'is', 'very', 'different', ',', 'shorn', 'of', 'any', 'honour', '.']
&              => anddred onedrddredyo || and 
                  ['science', '<SAMPLE>', 'justice', ',', '36', '(', '1', ')', ',', '51', '-', '54', '.']
Veach          => veach seniactyscerct || veach 
                  ['<SAMPLE>', 'had', '128', 'rbis', '(', '4th', 'in', 'the', 'al', ')', 'and', 'hit', '.', 

In [163]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.3, lr=0.001)

Saved model to data/models/whole_gen_13_only_chars_and_attn/250000_(EncoderRNN/DecoderRNN)
Accuracy: 0.40% (      40/   10000)
250040   5% (   9m 0s)   0.637   |   0.00: & -> and (✓) 
260040  10% ( 14m 41s)   0.555   |   0.00: centre -> center (✓) 
270040  15% ( 19m 42s)   0.565   |   0.00: Originally -> originally (✓) (forcing)
280040  20% ( 24m 24s)   0.528   |   0.00: NJ -> n j (✓) 
290040  25% ( 29m 13s)   0.532   |   2.95: 1813 -> eighteenn thirty  (✗: eighteen thirteen) 
Saved model to data/models/whole_gen_13_only_chars_and_attn/300000_(EncoderRNN/DecoderRNN)
Accuracy: 0.70% (      70/   10000)
300040  30% ( 37m 32s)   0.578   |   0.00: & -> and (✓) 
310040  35% ( 42m 24s)   0.525   |   5.94: dimanche.ch -> dimanqh w dch ncoct h w (✗: d i m a n c h e dot c h) 


IndexError: list index out of range

In [208]:
state_dict_path = 'data/models/whole_gen_13_only_chars_and_attn/10_'
#state_dict_path = 'data/models/whole_gen_13_only_chars_and_attn/250000_'

decoder_rnn.load_state_dict(torch.load(state_dict_path + 'DecoderRNN'))
encoder_rnn.load_state_dict(torch.load(state_dict_path + 'EncoderRNN'))

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0.1, lr=0.001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.1, lr=0.0005)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.1, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0, lr=0.0005)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0.2, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()

In [ ]:
train_iterations(n_iters=400000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

### Show attention

In [ ]:
balanced_data.sample(5)

In [ ]:
balanced_data.groupby('class')['class'].count()
len(balanced_data)

In [ ]:
def balanced_data_randomize_long():
    global balanced_data, balanced_data_length, balanced_data_accessed_counter, balanced_data_randomize_freq
    
    bal_data = pd.concat([v.sample(min(2000, len(v))) for k, v in balanced_data_classes_select])
    long_data = sample_data[sample_data['before'].str.len()>8].sample(4000)
    elec_data = sample_data[sample_data['class']=='ELECTRONIC']
    let_long_data = sample_data[(sample_data['class'] == 'LETTERS') & (sample_data['before'].str.len() > 5)]
    balanced_data = pd.concat([bal_data, long_data, elec_data, let_long_data])#.drop_duplicates()
    balanced_data = balanced_data[~balanced_data.index.duplicated(keep='first')]
    
    balanced_data_length = len(balanced_data)
    balanced_data_randomize_freq = balanced_data_length * 0.5
    balanced_data_accessed_counter = 0

balanced_data_randomize = balanced_data_randomize_long
balanced_data_randomize()

In [ ]:
def debug_show_attention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure(figsize=(15,6))
    ax = fig.add_subplot(111)
    #cax = ax.matshow(attentions.numpy(), cmap='bone')
    cax = ax.matshow(attentions, cmap='bone')
    fig.colorbar(cax)
    # Set up axes
    #ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    #ax.set_xticklabels([''] + input_sentence + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)
    
    input_sentence = input_sentence + ['<EOS>']
    #inp_arr = ["{}\n{}".format(input_sentence[i], input_sentence[-1-i]) for i in range(len(input_sentence))]
    inp_arr = input_sentence
    ax.set_xticklabels([''] + inp_arr, rotation=0)

    # Show label at every tick
    ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(1))

    plt.show()
    
def debug_eval_sample_show_attention():
    
    sample_row = balanced_data_sample_row()
    #sample_row = balanced_data[balanced_data['before'].str.len()>15].sample(1).iloc[0]
    sample = sample_row['before'], sample_row['a_word_ind'], sample_row['class'], sample_row['sentence'].split(' ')

    output, decoded_output, decoder_attns_arr, sample = test_model_single_sample(None, 
                                                            return_more=True, sample=sample)
    print('input:  ', sample[0])
    print('output: ', decoded_output)
    print('target:   ', ' '.join([words_after_common[w] for w in sample[1][:-1]]))

    attns = np.array([arr.data[0].cpu().numpy() for arr in decoder_attns_arr])

    debug_show_attention(list(sample[0]), decoded_output, attns)
    #plt.matshow(attns)

debug_eval_sample_show_attention()

In [ ]:
balanced_data.groupby('class')['class'].unique()

In [ ]:
def test_in_categories(iter_len = 1000):
    wrong_preds = {}
    for cat in categories_all:
        tmp_data = sample_data[sample_data['class'] == cat].sample(iter_len)
        correct_n = 0
        wrong_preds_arr = []

        for _ in range(iter_len):
            sample_row = tmp_data.iloc[_]
            sample = sample_row['before'], sample_row['a_word_ind'], sample_row['class'], sample_row['sentence']

            output, t1, sample_target, t2 = test_model_single_sample(None, sample=sample)
            if output == sample_target:
                correct_n += 1
            else:
                wrong_preds_arr.append([sample_target, output])

        print("{:>10}: {:>5d}/{:>5d} ({:>4.0%})".format(cat, correct_n, iter_len, correct_n/iter_len))
        wrong_preds[cat] = wrong_preds_arr
    return wrong_preds

In [ ]:
wrong_preds = test_in_categories(3000)

In [ ]:
wrong_preds['LETTERS']

In [ ]:
# With training longer words
wrong_preds = test_in_categories()